<a href="https://colab.research.google.com/github/ImranRiazChohan/EyeGazeMovementDetection/blob/main/TrainingEyeGazeMovementClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import cv2
import tensorflow as tf
from tensorflow.keras.layers import Dense,Flatten,Dropout,Conv2D,MaxPooling2D
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.optimizers import Adam,RMSprop
from tensorflow.keras.losses import categorical_crossentropy,sparse_categorical_crossentropy
from tensorflow.keras.metrics import Accuracy
import matplotlib.pyplot as plt
import os
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.vgg16 import VGG16 
from google.colab.patches import cv2_imshow

In [3]:
#Read Data From Directory
Orignal_dir="/content/drive/MyDrive/Internship_TechroticsLab/EyeGazeMovementDetector/EyeData"
Train_data_dir=os.path.join(Orignal_dir,"train")
train_center=os.path.join(Train_data_dir,"center")
train_left=os.path.join(Train_data_dir,"left")
train_right=os.path.join(Train_data_dir,"right")

In [5]:
#left_data=os.listdir(train_left)
#center_data=os.listdir(train_center)
#right_data=os.listdir(train_right)

In [6]:
#print(" Left Data",len(left_data),"\n","Right Data",
#len(right_data),"\n","Center Data", 
#len(center_data))

In [7]:
#train_left_img=[]
#for i in range(len(left_data)):
#  path=train_left+"/"+left_data[i]
#  img=cv2.imread(path)
#  cvt_img=cv2.resize(img,(64,56),3)
#  train_left_img.append(cvt_img)

In [8]:
#train_left_img[0].shape

In [9]:
#train_right_img=[]
#for i in range(len(right_data)):
#  path=train_right+"/"+right_data[i]
#  img=cv2.imread(path)
#  cvt_img=cv2.resize(img,(64,56),3)
#  train_right_img.append(cvt_img)


In [10]:
#train_center_img=[]
#for i in range(len(center_data)):
#  path=train_center+"/"+center_data[i]
#  img=cv2.imread(path)
#  cvt_img=cv2.resize(img,(64,56),3)
#  train_center_img.append(cvt_img)

In [11]:
#for i in range(0,5):
#  cv2_imshow(train_left_img[i])
#  print(train_left_img[i].shape)

In [12]:
#for i in range(0,5):
#  cv2_imshow(train_right_img[i])
#  print(train_right_img[i].shape)

In [13]:
#for i in range (0,5):
#  cv2_imshow(train_center_img[i])
#  print(train_center_img[i].shape)

In [22]:
test_data_dir=os.path.join(Orignal_dir,"test")
test_center_data=os.path.join(test_data_dir,"center")
test_left_data=os.path.join(test_data_dir,"left")
test_right_data=os.path.join(test_data_dir,"right")

In [15]:
#test_left=os.listdir(test_left_data)
#test_right=os.listdir(test_right_data)
#est_center=os.listdir(test_center_data)

In [16]:
#print("left:{}\nright:{}\ncenter:{}".format(len(test_left),len(test_right),len(test_center)))

In [17]:
#test_center_img=[]
#test_right_img=[]
#test_left_img=[]

#for i in range(len(test_center)):
#  path=test_center_data+"/"+test_center[i]
#  img_center=cv2.imread(path)
#  resize_img=cv2.resize(img_center,(64,56),3)
#  test_center_img.append(resize_img)
#for i in range(len(test_left)):
#  path=test_left_data+"/"+test_left[i]
#  img_left=cv2.imread(path)
#  resize_img=cv2.resize(img_left,(64,56),3)
#  test_left_img.append(resize_img)
#for i in range(len(test_right)):
#  path=test_right_data+"/"+test_right[i]
#  img_right=cv2.imread(path)
#  resize_img=cv2.resize(img_right,(64,56),3)
#  test_right_img.append(resize_img)


In [18]:
#path=test_center_data+"/"+test_center[0]
#img=cv2.imread(path)
#img.shape

In [19]:
train_data_gen=ImageDataGenerator(rotation_range=10,
                                  rescale=1./255,
                                  width_shift_range=0.1,
                                  height_shift_range=0.1,
                                  shear_range=0.1,
                                  zoom_range=0.1,
                                  fill_mode="nearest")
test_data_gen=ImageDataGenerator(rescale=1./255)
valid_data_gen=ImageDataGenerator(rescale=1./255)

In [20]:
#train_data_gen.flow_from_directory(Train_data_dir)
#test_data_gen.flow_from_directory(test_data_dir)

In [23]:
train_set=train_data_gen.flow_from_directory(Train_data_dir,target_size=(64,56),batch_size=16,color_mode="grayscale",class_mode="categorical")
test_set=test_data_gen.flow_from_directory(test_data_dir,target_size=(64,56),batch_size=16,color_mode="grayscale",class_mode="categorical")

Found 3622 images belonging to 3 classes.
Found 831 images belonging to 3 classes.


In [24]:
train_set.class_indices

{'center': 0, 'left': 1, 'right': 2}

In [31]:
model=Sequential()
model.add(Conv2D(32,kernel_size=(3,3),strides=1,padding="same",input_shape=(64,56,1),activation="relu"))
model.add(MaxPooling2D((2,2)))

model.add(Conv2D(64,kernel_size=(3,3),strides=1,padding="same",activation="relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D((2,2)))

model.add(Conv2D(128,kernel_size=(3,3),strides=1,padding="same",activation="relu"))
model.add(MaxPooling2D((2,2)))

model.add(Conv2D(256,kernel_size=(3,3),strides=1,padding="same",activation="relu"))
model.add(MaxPooling2D((2,2)))
model.add(Flatten())

model.add(Dense(512,activation="relu"))
model.add(Dropout(0.4))

model.add(Dense(512,activation="relu"))
model.add(Dropout(0.4))


model.add(Dense(3,activation="softmax"))

In [32]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_5 (Conv2D)           (None, 64, 56, 32)        320       
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 32, 28, 32)       0         
 2D)                                                             
                                                                 
 conv2d_6 (Conv2D)           (None, 32, 28, 64)        18496     
                                                                 
 batch_normalization_1 (Batc  (None, 32, 28, 64)       256       
 hNormalization)                                                 
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 16, 14, 64)       0         
 2D)                                                             
                                                      

In [33]:
model.compile(optimizer="Adam",loss="categorical_crossentropy",metrics=["acc"])

In [34]:
callback=EarlyStopping(monitor='val_loss',patience=2)

In [35]:
model.fit(train_set,epochs=8,validation_data=test_set,callbacks=[callback])

Epoch 1/8
227/227 [==============================] - 1459s 6s/step - loss: 0.1270 - acc: 0.9586 - val_loss: 0.6517 - val_acc: 0.9591
Epoch 2/8
227/227 [==============================] - 14s 63ms/step - loss: 0.0439 - acc: 0.9876 - val_loss: 0.0576 - val_acc: 0.9964
Epoch 3/8
227/227 [==============================] - 14s 62ms/step - loss: 0.0344 - acc: 0.9912 - val_loss: 0.0120 - val_acc: 0.9976
Epoch 4/8
227/227 [==============================] - 14s 61ms/step - loss: 0.0355 - acc: 0.9912 - val_loss: 0.0505 - val_acc: 0.9807
Epoch 5/8
227/227 [==============================] - 14s 61ms/step - loss: 0.0206 - acc: 0.9956 - val_loss: 0.0912 - val_acc: 0.9928


In [39]:
model.save("/content/drive/MyDrive/Internship_TechroticsLab/eyegaze.h5")